In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('ds_salaries.csv')
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [3]:
country_codes = pd.read_csv('countries_codes_and_coordinates.csv')
country_codes.head(5)
country_codes.drop(columns=['Numeric code', 'Latitude (average)', 'Longitude (average)'], inplace=True)
country_codes.head(5)

,Country,Alpha-2 code,Alpha-3 code
0,Afghanistan,"""AF""","""AFG"""
1,Albania,"""AL""","""ALB"""
2,Algeria,"""DZ""","""DZA"""
3,American Samoa,"""AS""","""ASM"""
4,Andorra,"""AD""","""AND"""


## Splitting the Data

In [4]:
df_train, df_test = train_test_split(df)

In [5]:
print('Training set: \n', "Rows: ", df_train.shape[0], "\tColumns: ", df_train.shape[1])
print('Test set: \n', "Rows: ", df_test.shape[0], "\tColumns: ", df_test.shape[1])

Training set: 
 Rows:  2816 	Columns:  11
Test set: 
 Rows:  939 	Columns:  11


## Data Exploration

### General Data 

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2816 entries, 294 to 745
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           2816 non-null   int64 
 1   experience_level    2816 non-null   object
 2   employment_type     2816 non-null   object
 3   job_title           2816 non-null   object
 4   salary              2816 non-null   int64 
 5   salary_currency     2816 non-null   object
 6   salary_in_usd       2816 non-null   int64 
 7   employee_residence  2816 non-null   object
 8   remote_ratio        2816 non-null   int64 
 9   company_location    2816 non-null   object
 10  company_size        2816 non-null   object
dtypes: int64(4), object(7)
memory usage: 264.0+ KB


### Making minor adjustments to the columns

In [7]:
df_train.columns = df_train.columns.str.replace('_', ' ')
df_test.columns = df_test.columns.str.replace('_', ' ')
df_train.columns

Index(['work year', 'experience level', 'employment type', 'job title',
       'salary', 'salary currency', 'salary in usd', 'employee residence',
       'remote ratio', 'company location', 'company size'],
      dtype='object')

### Checking for null values

In [8]:
df_train.isnull().sum()

work year             0
experience level      0
employment type       0
job title             0
salary                0
salary currency       0
salary in usd         0
employee residence    0
remote ratio          0
company location      0
company size          0
dtype: int64

### Getting Cat and Num columns

In [9]:
def get_catcolumns(dataframe):
    categorical_columns = []
    
    for i in dataframe.columns:
        if(dataframe[i].dtype != np.float64 and dataframe[i].dtype != np.int64):
            categorical_columns.append(i)

    return categorical_columns

def get_numcolumns(dataframe):
    numerical_columns = []
    
    for i in dataframe.columns:
        if(dataframe[i].dtype == np.float64 or dataframe[i].dtype == np.int64):
            numerical_columns.append(i)
    
    return numerical_columns
cat_columns = get_catcolumns(df_train)
num_columns = get_numcolumns(df_train)
print('Cat:',cat_columns, '\n Num:', num_columns)

Cat: ['experience level', 'employment type', 'job title', 'salary currency', 'employee residence', 'company location', 'company size'] 
 Num: ['work year', 'salary', 'salary in usd', 'remote ratio']


In [10]:
pd.set_option("display.max_rows", None)
df_train['company location'].value_counts()

US    2297
GB     128
CA      67
ES      53
IN      39
DE      38
FR      25
PT      13
BR      11
AU      10
NL      10
GR       9
MX       7
IE       5
NG       5
AT       5
SG       5
PK       4
TR       4
PR       4
PL       4
AS       3
DK       3
LV       3
UA       3
LU       3
CO       3
FI       3
JP       3
AE       3
IT       3
ID       2
EE       2
CZ       2
SE       2
CF       2
BE       2
IL       2
KE       2
GH       2
CH       2
MA       1
TH       1
NZ       1
HU       1
CN       1
SI       1
BS       1
BA       1
AL       1
PH       1
CL       1
AR       1
EG       1
LT       1
VN       1
MT       1
RO       1
DZ       1
RU       1
AM       1
IR       1
MY       1
SK       1
Name: company location, dtype: int64

## EDA

In [11]:
fig = px.choropleth(df, locations="iso_alpha",
                    color="lifeExp", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

ValueError: Value of 'hover_name' is not the name of a column in 'data_frame'. Expected one of ['work_year', 'experience_level', 'employment_type', 'job_title', 'salary', 'salary_currency', 'salary_in_usd', 'employee_residence', 'remote_ratio', 'company_location', 'company_size'] but received: country

In [ ]:
mean_salaries_exp = df_train.groupby('experience level')['salary in usd'].mean()
mean_salaries_exp_graph = px.bar(mean_salaries_exp,
       labels={
           'value':'USD',
           'experience level':'Experience Level'
       },
      title='Salaries per Experience')

mean_salaries_exp_graph.show()

Graph to show the change of salaries over time for each job title

In [ ]:
df_train['work_year'] = df_train['work_year'].apply(str)
df_test['work_year'] = df_test['work_year'].apply(str)

grouped = df_train.groupby(['job_title', 'work_year'])['salary_in_usd'].mean()
highest_paying_jobs = grouped.groupby('job_title').max().sort_values(ascending=False)[:5].index
filtered = grouped.loc[highest_paying_jobs]
unstacked = filtered.unstack(level=0)
unstacked

px.line(unstacked,labels={
                     "job_title": "Job Title",
                     "work_year": "Years",
                     "value": "Salary in USD"},
       title="Salaries Over the Years")

In [ ]:
y_train = df_train['salary_in_usd']
x_train = df_train.drop(columns=['salary_in_usd'])

y_test = df_test['salary_in_usd']
x_test = df_test.drop(columns=['salary_in_usd'])

In [ ]:
px.box(y_train)


## Data Preparation and Feature Engineering for the models

### Libraries
Since we have

This is a change